# Exercise: Analyzing Chinook Database

Preparation I've done:
 - Retrieve the dataset and load it
 - Load the %sql extension and point it at the database
 - Display the tables and an example query

Additional steps you might take:
 - Add libraries for visualization (matplotlib, seaborn, plotly)
 - Add libraries for statistics (numpy)
 - Explore the dataset using SQL and/or pandas

----

1. Retrieve a list of all the tracks in the database, displaying only the track name and the name of the album it belongs to. Limit the result to the first 5 rows.
   > Operations: `SELECT`
2. Find the total number of customers from each country. Display the country name and the corresponding count. Order the results by the count in descending order.
   > Operations: `SELECT`, `COUNT`, `GROUP BY`, `ORDER BY`
3. Identify the top 5 genres with the highest number of tracks. Display the genre name along with the total number of tracks for each genre.
   > Operations: `SELECT`, `COUNT`, `GROUP BY`, `ORDER BY`
4. Determine the average invoice total for each customer, considering both the album and individual track purchases. Display the customer's first and last name along with the average invoice total. Order the results by the average invoice total in descending order.
   > Operations: `SELECT`, `AVG`, `JOIN`, `GROUP BY`, `ORDER BY`
5. Identify the customer who spent the most on music purchases. Display the customer's first and last name, along with the total amount spent.
   > Operations: `SELECT`, `SUM`, `JOIN`, `GROUP BY`, `ORDER BY`, `LIMIT`

In [67]:
# Load chinook dataset and query it using SQL magic into pandas dataframes
%pip install pandas

import pandas as pd

# Install ipython-sql if not already installed
%pip install ipython-sql

# Load SQL extension
%load_ext sql

# Import sqlite3
import sqlite3

# Load data
conn = sqlite3.connect("chinook.sqlite")

# Tell %sql about the database
%sql sqlite:///chinook.sqlite

# List tables in database
query = "SELECT name FROM sqlite_master WHERE type='table';"

# Read data into a Pandas DataFrame
tables = %sql $query

# Print head
display(tables)

# Query to get the first 5 rows of the `albums` table
result = %sql SELECT * FROM albums LIMIT 5;

# Display query result, note that Pandas DataFrame is returned!
display(result)



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
The sql extension is already loaded. To reload it, use:
  %reload_ext sql
 * sqlite:///chinook.sqlite
Done.


name
albums
sqlite_sequence
artists
customers
employees
genres
invoices
invoice_items
media_types
playlists


 * sqlite:///chinook.sqlite
Done.


AlbumId,Title,ArtistId
1,For Those About To Rock We Salute You,1
2,Balls to the Wall,2
3,Restless and Wild,2
4,Let There Be Rock,1
5,Big Ones,3


In [104]:
#1. Retrieve a list of all the tracks in the database, displaying only the track name and the name of the album it belongs to. 
#Limit the result to the first 5 rows.

  # > Operations: `SELECT`

track_list2= %sql SELECT Name, Albumid FROM tracks \
LIMIT 5;
display (track_list2)





 * sqlite:///chinook.sqlite
Done.


Name,AlbumId
For Those About To Rock (We Salute You),1
Balls to the Wall,2
Fast As a Shark,3
Restless and Wild,3
Princess of the Dawn,3


In [105]:
#Extra steps to build on question #1


#I then decided to build on question #1 by displaying the album NAME instead of the ID
#This requires a join, as Album name is in the albums table, wheras the track name is in the tracks table

#We note here that all of the tracks are coming from the same album, which is in contrast to the first part of the exercise

track_list= %sql SELECT tracks.Name AS Track_Name, albums.Title AS Album_Name \
FROM tracks \
JOIN albums ON tracks.Albumid=albums.Albumid \
LIMIT 5;

display (track_list)

 * sqlite:///chinook.sqlite
Done.


Track_Name,Album_Name
For Those About To Rock (We Salute You),For Those About To Rock We Salute You
Put The Finger On You,For Those About To Rock We Salute You
Let's Get It Up,For Those About To Rock We Salute You
Inject The Venom,For Those About To Rock We Salute You
Snowballed,For Those About To Rock We Salute You


In [107]:
track_list= %sql SELECT tracks.Name AS Track_Name, tracks.AlbumId, albums.Title AS Album_Name \
FROM tracks \
JOIN albums ON tracks.Albumid=albums.Albumid \
GROUP BY tracks.AlbumId \
LIMIT 20; #If we group by album ID and extend the number of observations, we are able to get a list of tracks from each album

display (track_list)


 * sqlite:///chinook.sqlite
Done.


Track_Name,AlbumId,Album_Name
For Those About To Rock (We Salute You),1,For Those About To Rock We Salute You
Balls to the Wall,2,Balls to the Wall
Fast As a Shark,3,Restless and Wild
Go Down,4,Let There Be Rock
Walk On Water,5,Big Ones
All I Really Want,6,Jagged Little Pill
We Die Young,7,Facelift
Desafinado,8,Warner 25 Anos
Enter Sandman,9,Plays Metallica By Four Cellos
Cochise,10,Audioslave


In [118]:
#2. Find the total number of customers from each country. 
#Display the country name and the corresponding count. 
#Order the results by the count in descending order.
   #> Operations: `SELECT`, `COUNT`, `GROUP BY`, `ORDER BY`

#Create a new label for the customer count, specify the main table, Want to order by the customer count

cust_nation = %sql SELECT Country, COUNT(CustomerId) AS Num_Customers\
FROM customers\
GROUP BY Country\
ORDER BY Num_Customers DESC;

display(cust_nation)



 * sqlite:///chinook.sqlite
Done.


Country,Num_Customers
USA,13
Canada,8
France,5
Brazil,5
Germany,4
United Kingdom,3
Portugal,2
India,2
Czech Republic,2
Sweden,1


In [119]:
#3. Identify the top 5 genres with the highest number of tracks. 
#Display the genre name along with the total number of tracks for each genre.
  # > Operations: `SELECT`, `COUNT`, `GROUP BY`, `ORDER BY`

#Count the number of track ids and assign a new label
#Join on genreID which is present in both tables. Group by genre and order by the number of tracks

genre_track = %sql SELECT genres.Name, COUNT(tracks.Trackid) AS Num_Tracks \
FROM genres \
JOIN tracks ON genres.Genreid=tracks.Genreid \
GROUP BY genres.Name \
ORDER BY COUNT(tracks.Trackid) \
DESC Limit 5;
display(genre_track)





 * sqlite:///chinook.sqlite
Done.


Name,Num_Tracks
Rock,1297
Latin,579
Metal,374
Alternative & Punk,332
Jazz,130


In [126]:
#4. Determine the average invoice total for each customer, 
#considering both the album and individual track purchases. 
#Display the customer's first and last name along with the average invoice total. 
#Order the results by the average invoice total in descending order.
   #> Operations: `SELECT`, `AVG`, `JOIN`, `GROUP BY`, `ORDER BY`

avg_invoice = %sql SELECT ROUND(AVG(invoices.Total),2) AS AverageInvoiceTotal, customers.FirstName, customers.LastName \
FROM customers \
JOIN invoices ON customers.CustomerId=invoices.CustomerId \
GROUP BY customers.FirstName, customers.LastName \
ORDER BY AverageInvoiceTotal DESC;

display (avg_invoice)



 * sqlite:///chinook.sqlite
Done.


AverageInvoiceTotal,FirstName,LastName
7.09,Helena,Holý
6.8,Richard,Cunningham
6.66,Luis,Rojas
6.52,Hugh,O'Reilly
6.52,Ladislav,Kovács
6.23,Frank,Ralston
6.23,Fynn,Zimmermann
6.23,Julia,Barnett
6.11,Puja,Srivastava
6.09,Astrid,Gruber


In [122]:
#5. Identify the customer who spent the most on music purchases. 
#Display the customer's first and last name, 
#along with the total amount spent.
   #> Operations: `SELECT`, `SUM`, `JOIN`, `GROUP BY`, `ORDER BY`, `LIMIT`

cust_pay = %sql SELECT SUM(invoice_items.UnitPrice * invoice_items.Quantity) AS Total_cost, customers.FirstName, customers.LastName \
FROM customers \
JOIN invoices ON customers.CustomerId = invoices.CustomerId \
JOIN invoice_items ON invoices.InvoiceId = invoice_items.InvoiceId \
GROUP BY customers.FirstName, customers.LastName \
ORDER BY Total_cost DESC LIMIT 1;

display(cust_pay)

 * sqlite:///chinook.sqlite
Done.


Total_cost,FirstName,LastName
49.62,Helena,Holý
